# Projet LEYENDA
## Livrable 1 - Classification Binaire

#### Importation des bibliothèques

In [3]:
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import tensorflow as tf
import pathlib
import zipfile

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

### Préparation des images
#### Dézip des fichiers

In [4]:
repertoire = './'
dossier_dataset = './Data'
if not os.path.exists(dossier_dataset):
    os.makedirs(dossier_dataset)
    
for fichier in os.listdir(repertoire):
    chemin_fichier = os.path.join(repertoire,fichier)
    if zipfile.is_zipfile(chemin_fichier):
        with zipfile.ZipFile(chemin_fichier, 'r') as zip_ref:
            zip_ref.extractall(dossier_dataset)
            print(f'Fichier {fichier} décompressé dans le dossier Dataset.')
print('Tous les fichiers ZIP ont été décompressés dans le dossier Dataset')

Fichier Dataset Livrable 1 - Photo.zip décompressé dans le dossier Dataset.
Fichier Dataset Livrable 1 - Text.zip décompressé dans le dossier Dataset.
Fichier Dataset Livrable 1 - Sketch.zip décompressé dans le dossier Dataset.
Fichier Dataset Livrable 1 - Schematics.zip décompressé dans le dossier Dataset.
Tous les fichiers ZIP ont été décompressés dans le dossier Dataset


#### Emplacement du dataset

In [8]:
datapath = "./Data/"
data_dir = pathlib.Path(datapath)

#### Mise à l'échelle des images

In [ ]:
import os
from PIL import Image

# Dossier contenant les images
dossier_images = ----------------

# Dictionnaire pour stocker le nombre d'images par dimension
dimensions_images = {}

# Parcourir le dossier
for fichier in os.listdir(dossier_images):
    chemin_fichier = os.path.join(dossier_images, fichier)

    # Vérifier si le fichier est une image
    if not os.path.isfile(chemin_fichier) or not fichier.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp')):
        continue

    # Ouvrir l'image avec Pillow
    try:
        image = Image.open(chemin_fichier)
    except Exception as e:
        print(f"Impossible d'ouvrir l'image {chemin_fichier}: {str(e)}")
        continue

    # Obtenir les dimensions de l'image
    largeur, hauteur = image.size
    dimensions = (largeur, hauteur)

    # Compter le nombre d'images avec les mêmes dimensions
    if dimensions in dimensions_images:
        dimensions_images[dimensions] += 1
    else:
        dimensions_images[dimensions] = 1

# Afficher le résultat
for dimensions, nombre in dimensions_images.items():
    print(f"Dimensions {dimensions}: {nombre} images")

# Enregistrez les résultats dans un fichier texte si nécessaire
with open("resultats_dimensions.txt", "w") as fichier_resultats:
    for dimensions, nombre in dimensions_images.items():
        fichier_resultats.write(f"Dimensions {dimensions}: {nombre} images\n")


Les images n'ont pas de dimension standard, il faut donc les traiter pour obtenir des images aux dimensions identiques afin de pouvoir les utiliser dans nos modèles futurs

target_size = (224, 224)
for folder_name in os.listdir(base_path):
    folder_path = os.path.join(base_path, folder_name)

    if os.path.isdir(folder_path):
        for image_name in os.listdir(folder_path):
            image_path = os.path.join(folder_path, image_name)

            if image_name.endswith(('.jpg', '.jpeg', '.png', '.bmp')):
                image = Image.open(image_path)

                # Calculez les coordonnées de recadrage centré pour conserver les parties importantes
                width, height = image.size
                left = (width - target_size[0]) / 2
                top = (height - target_size[1]) / 2
                right = (width + target_size[0]) / 2
                bottom = (height + target_size[1]) / 2

                # Effectuez le recadrage centré
                image = image.crop((left, top, right, bottom))

                # Redimensionnez l'image à la taille cible
                image = image.resize(target_size, Image.ANTIALIAS)

                image = np.array(image)  # Convertir en tableau NumPy
                image_paths.append(image_path)
                labels.append(folder_name)

## ------------------------------------------------------------------------

In [9]:
image_h = 360
image_w = 360
batch_s = 32

In [11]:
# Le train_set
train_set = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir = "./Data",
    validation_split=  0.2,
    subset =  "training",
    seed=42,
    image_size=(image_h, image_w),
    batch_size=batch_s
)
# Le test_set
test_set = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split=  0.2,
    subset =  "validation",
    seed=42,
    image_size=(image_h, image_w),
    batch_size=batch_s
)

TypeError: image_dataset_from_directory() missing 1 required positional argument: 'directory'

In [ ]:
class_names = train_set.class_names
print(class_names)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(8, 8))
for images, labels in train_set.take(1):
    for i in range(9):
        ax =  plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(class_names[labels[i]])
        plt.axis("off")

In [ ]:
print(type(train_set))
images, labels =  next(iter(train_set))
print(images.shape)
print(labels.shape)

In [ ]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

train_set = train_set.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
test_set = test_set.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
num_classes = 5 # Nombre de classes et donc aussi nombre de neurones dans la dernière couche
model = Sequential()

In [ ]:
model.add(layers.experimental.preprocessing.Rescaling(1./255))

In [ ]:
# Couche de convolution
model.add(layers.Conv2D(16, (3, 3), activation='relu', padding='same'))
# Couche de pooling
model.add(layers.MaxPooling2D((2, 2)))

In [ ]:
# Bloc convolutif ou la taille du filtre est de (32, 3)
model.add(layers.Conv2D(32, (3, 3), activation='relu', padding='same'))

# Bloc convolutif ou la taille du filtre est de (64, 3)
model.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same'))

# Applatissement de la couche
model.add(layers.Flatten())

# Couche entièrement connectée (couche dense)
model.add(layers.Dense(128, activation='relu'))

# Couche entièrement connectée retournant le résultat de la classification
model.add(layers.Dense(num_classes))

model.build((None, image_h, image_w, 3))
model.summary()

In [ ]:
model.compile(optimizer =  'adam',
              loss =  tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
model.summary()

In [ ]:
from keras.src.layers.preprocessing.image_preprocessing import HORIZONTAL_AND_VERTICAL
from keras.src.layers.preprocessing.image_preprocessing import RandomFlip
from keras.src.layers.preprocessing.image_preprocessing import RandomRotation
from keras.src.layers.preprocessing.image_preprocessing import RandomZoom
from keras import Sequential

data_augmentation = Sequential(
    [
        RandomFlip(
            mode=HORIZONTAL_AND_VERTICAL,
            input_shape=(image_h, image_w, 3)),
        RandomRotation(
            factor=0.18,
            fill_mode='reflect',
            interpolation='bilinear',
            seed=None,
            fill_value=0.0),
        RandomZoom(
            height_factor=0.1,
            width_factor=None,
            fill_mode='reflect',
            interpolation='bilinear',
            seed=None,
            fill_value=0.0)
    ]
)

In [ ]:
# Le modèle
epochs = 8
complete_model =  Sequential([
    layers.experimental.preprocessing.Rescaling(1./255),
    data_augmentation,
    layers.Conv2D(16, (3, 3), activation='relu', padding='same'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(32, (3, 3), activation='relu', padding='same'),
    layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
    layers.Dropout(0.5),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(num_classes)
])
complete_model.build((None, image_h, image_w, 3))
# Compilation du modèle
complete_model.compile(optimizer =  'adam',
                       loss =  tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                       metrics=['accuracy'])
# Résumé du modèle
complete_model.summary()
# Enrainement du modèle
history =  complete_model.fit(
    train_set,
    validation_data=test_set,
    epochs=epochs
)
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(16, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()